In [1]:
import numpy as np
import random
from numpy import inf
from operator import itemgetter


In [2]:
cost_matrix = np.array([[inf, 1,   inf, inf, inf],
                        [2,   inf, 3,   1,   inf],
                        [inf, 2,   inf, 1,   inf],
                        [inf, 1,   inf, inf, 2  ],
                        [inf, inf, 2,   inf, inf]])

for i, line in enumerate(cost_matrix):
    pass

starting_points = [0,4]

number_of_points = len(cost_matrix)

start_pos=[0,4]

number_of_solutions = 1000
le = 10
lb = 500
ne = 200
nb = 100
lt = 200
i_max = 10000

solutions = []
solutions_scores = []
solutions_alive_time = []



def generate_random_solution(number_of_points):
    result = list(range(number_of_points))
    random.shuffle(result)
    return np.array(result)
    

# print(generate_random_solution(number_of_points))

def eval_solution(path):
    time_multiplier = 1
    missed_path_multiplier = 1000
    out_of_bounds_multiplier = 10000

    fitness = 0

    # czas przejścia
    for i in range(len(path)-1):
        fitness += time_multiplier*cost_matrix[path[i],path[i+1]]

    # kara za pominięcie szlaku
    all_paths = np.ma.masked_invalid(cost_matrix).nonzero()  # TODO wyjebać to poza funkcje
    all_paths = list(zip(*all_paths)) # transpose
    all_paths = list({*map(tuple, map(sorted, all_paths))}) # remove duplicates

    # print(all_paths)

    number_of_missed_paths = 0
    for a,b in all_paths:
        if (repr(path).count(f"{a}, {b}") == 0 and repr(path).count(f"{b}, {a}") == 0):
            number_of_missed_paths+=1

    fitness += missed_path_multiplier*number_of_missed_paths

    # kara za wyjście poza szlak
    number_of_oob_paths = 0
    for i in range(len(path)-1):
        if (cost_matrix[path[i], path[i+1]]==inf):
            number_of_oob_paths += 1

    fitness += out_of_bounds_multiplier*number_of_oob_paths

    return fitness

# print(eval_solution([0,1,2,3,4]))


def generate_neighbour(path, number_of_points, cost_matrix, starting_points):
    new_neighbour = []
    while(len(new_neighbour)==0):
        rand = random.randint(1, 3) 
        position = random.randint(0, len(path)-1)
        position2 = random.randint(0, len(path)-1)
        new_value = random.randint(0, number_of_points-1)
        if rand == 1:
            new_neighbour = np.append(path, new_value)

        elif rand ==2:
            path[position], path[position2] = path[position2], path[position]
            new_neighbour = np.array(path)

        elif rand == 69:
            all_paths = np.ma.masked_invalid(cost_matrix).nonzero()  # TODO wyjebać to poza funkcje
            all_paths = list(zip(*all_paths))  # transpose
            # all_paths = list({*map(tuple, map(sorted, all_paths))})  # remove duplicates

            target_value = cost_matrix[position]
            if(target_value != inf):
                new_neighbour = [x for x in all_paths if any(target_value in x)]

        else:
            path[position]=new_value
            new_neighbour = np.array(path)
    return new_neighbour

            

    

for i in range(number_of_solutions):
    temp_solution = generate_random_solution(number_of_points)
    solutions.append(temp_solution)
    solutions_scores.append(eval_solution(temp_solution))
    solutions_alive_time.append(0)

solutions_scores, solutions, solutions_alive_time = map(list, zip(*sorted(zip(solutions_scores, solutions, solutions_alive_time), key=itemgetter(0))))

best_solution = solutions[0]
best_solution_score = solutions_scores[0]

iterations=0

while(iterations<i_max):

    best_neighbours = []
    best_neighbours_scores = []

    for i, current_solution in enumerate(solutions):
        neighbourhood = []
        if(i<le):
            for j in range(ne):
                neighbourhood.append(generate_neighbour(current_solution,number_of_points,cost_matrix))
                iterations+=1
        elif(i<lb):
            for j in range(nb):
                iterations += 1
                neighbourhood.append(generate_neighbour(current_solution, number_of_points, cost_matrix))
        else:
            break
        neighbourhood_scores = []
        for neighbour in neighbourhood:
            neighbourhood_scores.append(eval_solution(neighbour))
        # print(neighbourhood_scores, neighbourhood)
        neighbourhood_scores, neighbourhood = [list(x) for x in zip(*sorted(zip(neighbourhood_scores, neighbourhood), key=itemgetter(0)))]
        best_neighbours.append(neighbourhood[0])
        best_neighbours_scores.append(neighbourhood_scores[0])

    deleted_amount=0

    for i, current_neighbourhood in enumerate(neighbourhood):
        if(solutions_scores[i]<best_neighbours_scores[i]):
            solutions[i]=neighbourhood[i]
            solutions_scores[i]=list(best_neighbours_scores[i])
            solutions_alive_time[i]=0
        solutions_alive_time[i]+=1
        if(solutions_alive_time[i]>lt):
            deleted_amount += 1
            iterations+=1
            solutions[i] = generate_random_solution(number_of_points)
            solutions_scores[i] = eval_solution(solutions[i])
            solutions_alive_time[i] = 0

    solutions_scores, solutions, solutions_alive_time = map(list, zip(*sorted(zip(solutions_scores, solutions, solutions_alive_time), key=itemgetter(0))))

    if(best_solution_score>solutions_scores[0]):
        best_solution=solutions[0]
        best_solution_score=solutions_scores[0]


print(best_solution)
print(best_solution_score)


[0 0 3 0 1]
2006.0
